# Task 4 Extra

## Move and check all of the images 

In [2]:
import os
import shutil


In [23]:
import os
import shutil

def copy_root_masks(masks_dir, masks_destination):
    """
    Copies only the masks ending with '_root_mask.tif' from masks_dir to masks_destination.
    """
    os.makedirs(masks_destination, exist_ok=True)
    copied_count = 0  # Counter for debugging

    # Get all masks in the directory
    mask_files = os.listdir(masks_dir)

    for mask in mask_files:
        if mask.endswith("_root_mask.tif"):  # Check if the mask ends with '_root_mask.tif'
            mask_path = os.path.join(masks_dir, mask)
            dest_path = os.path.join(masks_destination, mask)
            shutil.copy(mask_path, dest_path)
            print(f"Copied root mask: {mask_path} -> {dest_path}")
            copied_count += 1

    print(f"Total root masks copied: {copied_count}")

def copy_corresponding_images(masks_destination, images_dir, image_destination):
    """
    Matches masks in masks_destination with corresponding images in images_dir
    and copies them to image_destination. The images are expected to have the same
    base name as the masks but with a .png extension and without mask-specific suffixes.
    """
    os.makedirs(image_destination, exist_ok=True)
    copied_count = 0  # Counter for debugging

    # Extract base names from mask files
    mask_files = os.listdir(masks_destination)
    base_names = {mask.rsplit('_', 2)[0] for mask in mask_files if mask.endswith("_root_mask.tif")}  # Extract unique base names

    # Match and copy corresponding images
    for base_name in base_names:
        image_name = f"{base_name}.png"  # Reconstruct the image filename
        image_path = os.path.join(images_dir, image_name)
        if os.path.exists(image_path):  # Check if the image exists
            dest_path = os.path.join(image_destination, image_name)
            shutil.copy(image_path, dest_path)
            print(f"Copied image: {image_path} -> {dest_path}")
            copied_count += 1
        else:
            print(f"Image not found for mask: {base_name}")

    print(f"Total images copied: {copied_count}")

# Example usage:

# Step 1: Copy only root masks
copy_root_masks(
    masks_dir=r"D:\Holland_Year_2\Block_B\Y2B_24\masks",
    masks_destination=r"D:\Holland_Year_2\Block_B\Task5_Patches\train_masks_corrected"
)

# Step 2: Copy corresponding images
copy_corresponding_images(
    masks_destination=r"D:\Holland_Year_2\Block_B\Task5_Patches\train_masks_corrected",
    images_dir=r"D:\Holland_Year_2\Block_B\Y2B_24\images",
    image_destination=r"D:\Holland_Year_2\Block_B\Task5_Patches\train_images_corrected"
)


Copied root mask: D:\Holland_Year_2\Block_B\Y2B_24\masks\train_Alican_230858_im1_root_mask.tif -> D:\Holland_Year_2\Block_B\Task5_Patches\train_masks_corrected\train_Alican_230858_im1_root_mask.tif
Copied root mask: D:\Holland_Year_2\Block_B\Y2B_24\masks\train_Alican_230858_im2_root_mask.tif -> D:\Holland_Year_2\Block_B\Task5_Patches\train_masks_corrected\train_Alican_230858_im2_root_mask.tif
Copied root mask: D:\Holland_Year_2\Block_B\Y2B_24\masks\train_Alican_230858_im3_root_mask.tif -> D:\Holland_Year_2\Block_B\Task5_Patches\train_masks_corrected\train_Alican_230858_im3_root_mask.tif
Copied root mask: D:\Holland_Year_2\Block_B\Y2B_24\masks\train_Alican_230858_im4_root_mask.tif -> D:\Holland_Year_2\Block_B\Task5_Patches\train_masks_corrected\train_Alican_230858_im4_root_mask.tif
Copied root mask: D:\Holland_Year_2\Block_B\Y2B_24\masks\train_Alican_230858_im5_root_mask.tif -> D:\Holland_Year_2\Block_B\Task5_Patches\train_masks_corrected\train_Alican_230858_im5_root_mask.tif
Copied roo

--- - - - - 

## Patches 

In [1]:
from patchify import patchify, unpatchify
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

### Cropping

In [3]:
import os
import numpy as np
import cv2
from PIL import Image
import matplotlib.pyplot as plt

# Define functions
def load_image(image_path):
    """ Load an image in grayscale. """
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if image is None:
        print(f"Failed to load image: {image_path}")
        return None
    return image

def detect_edges(image, max_size=2800):
    """ Detect edges to find the approximate square Petri dish. """
    blurred_image = cv2.GaussianBlur(image, (51, 51), 0)

    # Compute horizontal and vertical gradients
    sobel_x = cv2.Sobel(blurred_image, cv2.CV_64F, 1, 0, ksize=5)
    sobel_y = cv2.Sobel(blurred_image, cv2.CV_64F, 0, 1, ksize=5)

    # Calculate the gradient magnitude
    gradient_magnitude = cv2.magnitude(sobel_x, sobel_y)

    # Threshold to identify strong edges
    _, edges = cv2.threshold(gradient_magnitude, 50, 255, cv2.THRESH_BINARY)
    edges = edges.astype(np.uint8)

    # Find contours to identify the Petri dish
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Assume the largest square-like contour is the Petri dish
    max_contour = max(contours, key=cv2.contourArea)
    x, y, w, h = cv2.boundingRect(max_contour)

    # Ensure the crop is a square, adjust dimensions as needed
    side_length = max(w, h)
    side_length = min(side_length, max_size)  # Clamp to max_size

    center_x, center_y = x + w // 2, y + h // 2
    half_side = side_length // 2

    # Define new bounding box that is a square centered at the original bounding box
    new_x = max(center_x - half_side, 0)
    new_y = max(center_y - half_side, 0)
    new_w = new_h = min(side_length, min(image.shape[1] - new_x, image.shape[0] - new_y))

    return new_x, new_x + new_w, new_y, new_y + new_h

def crop_image(image, edges, max_size=2800):
    """ Crop and resize the image to ensure dimensions are below the threshold. """
    left, right, top, bottom = edges
    cropped = image[top:bottom, left:right]

    # Resize if necessary
    if cropped.shape[1] > max_size or cropped.shape[0] > max_size:
        cropped = cv2.resize(cropped, (min(cropped.shape[1], max_size), min(cropped.shape[0], max_size)))

    return cropped

def process_and_crop_images(original_dir, tif_dir, output_original_dir, output_tif_dir):
    """ Process and crop images in original and corresponding TIFF files the same way. """
    processed_files = []
    skipped_files = []

    for filename in os.listdir(original_dir):
        if filename.endswith(".png"):
            original_image_path = os.path.join(original_dir, filename)

            # Match TIFF file using improved logic
            base_name = filename.rsplit(".", 1)[0]  # Extract base name from image filename
            corresponding_tif_path = None
            for tif_file in os.listdir(tif_dir):
                tif_base_name = tif_file.replace("_root_mask", "").rsplit(".", 1)[0]  # Strip '_root_mask' and extension
                if tif_base_name == base_name:
                    corresponding_tif_path = os.path.join(tif_dir, tif_file)
                    break

            if not corresponding_tif_path:
                print(f"Corresponding TIFF file not found for {filename}")
                skipped_files.append(filename)
                continue

            # Load original and corresponding TIFF images
            original_image = load_image(original_image_path)
            try:
                tif_image = np.array(Image.open(corresponding_tif_path))
            except Exception as e:
                print(f"Failed to load TIFF file: {corresponding_tif_path} - {e}")
                skipped_files.append(filename)
                continue

            if original_image is None or tif_image is None:
                skipped_files.append(filename)
                continue

            # Detect edges and crop
            edges = detect_edges(original_image)
            cropped_original = crop_image(original_image, edges)
            cropped_tif = crop_image(tif_image, edges)

            # Save cropped images
            output_original_path = os.path.join(output_original_dir, filename)
            output_tif_path = os.path.join(output_tif_dir, os.path.basename(corresponding_tif_path))

            cv2.imwrite(output_original_path, cropped_original)
            Image.fromarray(cropped_tif).save(output_tif_path)

            print(f"Processed and saved: {filename} and corresponding mask")
            processed_files.append(filename)

    # Debugging information
    print("\nSummary of Processing:")
    print(f"Total files processed: {len(processed_files)}")
    print(f"Total files skipped: {len(skipped_files)}")
    if skipped_files:
        print("Skipped files:")
        for skipped in skipped_files:
            print(f"  - {skipped}")

# Define directories
original_dir = "D:/Holland_Year_2/Block_B/Task5_Patches/train_images_corrected"
tif_dir = "D:/Holland_Year_2/Block_B/Task5_Patches/train_masks_corrected"
output_original_dir = "D:/Holland_Year_2/Block_B/Task5_Patches/train_images_cropped_"
output_tif_dir = "D:/Holland_Year_2/Block_B/Task5_Patches/train_masks_cropped_"

# Ensure output directories exist
os.makedirs(output_original_dir, exist_ok=True)
os.makedirs(output_tif_dir, exist_ok=True)

# Process and crop images
process_and_crop_images(original_dir, tif_dir, output_original_dir, output_tif_dir)


Processed and saved: train_Alican_230858_im1.png and corresponding mask
Processed and saved: train_Alican_230858_im2.png and corresponding mask
Processed and saved: train_Alican_230858_im3.png and corresponding mask
Processed and saved: train_Alican_230858_im4.png and corresponding mask
Processed and saved: train_Alican_230858_im5.png and corresponding mask
Processed and saved: train_Alican_231131_im1.png and corresponding mask
Processed and saved: train_Alican_231131_im2.png and corresponding mask
Processed and saved: train_Alican_231131_im3.png and corresponding mask
Processed and saved: train_Alican_231131_im4.png and corresponding mask
Processed and saved: train_Alican_231131_im5.png and corresponding mask
Processed and saved: train_Alican_231648_im1.png and corresponding mask
Processed and saved: train_Alican_231648_im2.png and corresponding mask
Processed and saved: train_Alican_231648_im3.png and corresponding mask
Processed and saved: train_Alican_231648_im4.png and correspondi

In [2]:
import os
import numpy as np
import cv2
from PIL import Image
import matplotlib.pyplot as plt
from patchify import patchify

# Define functions
def load_image(image_path):
    """ Load an image in grayscale. """
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if image is None:
        print(f"Failed to load image: {image_path}")
        return None
    return image

def detect_edges(image):
    """ Detect edges to find the approximate square Petri dish. """
    blurred_image = cv2.GaussianBlur(image, (51, 51), 0)

    # Compute horizontal and vertical gradients
    sobel_x = cv2.Sobel(blurred_image, cv2.CV_64F, 1, 0, ksize=5)
    sobel_y = cv2.Sobel(blurred_image, cv2.CV_64F, 0, 1, ksize=5)

    # Calculate the gradient magnitude
    gradient_magnitude = cv2.magnitude(sobel_x, sobel_y)

    # Threshold to identify strong edges
    _, edges = cv2.threshold(gradient_magnitude, 50, 255, cv2.THRESH_BINARY)
    edges = edges.astype(np.uint8)

    # Find contours to identify the Petri dish
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Assume the largest square-like contour is the Petri dish
    max_contour = max(contours, key=cv2.contourArea)
    x, y, w, h = cv2.boundingRect(max_contour)

    # Ensure the crop is a square, adjust dimensions as needed
    side_length = max(w, h)
    center_x, center_y = x + w // 2, y + h // 2
    half_side = side_length // 2

    # Define new bounding box that is a square centered at the original bounding box
    new_x = max(center_x - half_side, 0)
    new_y = max(center_y - half_side, 0)
    new_w = new_h = min(side_length, min(image.shape[1] - new_x, image.shape[0] - new_y))

    return new_x, new_x + new_w, new_y, new_y + new_h

def crop_image(image, edges):
    """ Crop the image based on detected edges. """
    left, right, top, bottom = edges
    return image[top:bottom, left:right]

def padder(image, patch_size):
    """ Pad an image to make its dimensions divisible by the patch size. """
    h, w = image.shape[:2]
    height_padding = ((h // patch_size) + 1) * patch_size - h
    width_padding = ((w // patch_size) + 1) * patch_size - w

    top_padding = int(height_padding / 2)
    bottom_padding = height_padding - top_padding
    left_padding = int(width_padding / 2)
    right_padding = width_padding - left_padding

    padded_image = cv2.copyMakeBorder(image, top_padding, bottom_padding, left_padding, right_padding, cv2.BORDER_CONSTANT, value=[0, 0, 0])
    return padded_image

def patchify_images_and_masks(image_dir, mask_dir, output_image_dir, output_mask_dir, patch_size):
    """ Patchify cropped images and masks. """
    processed_files = []
    skipped_files = []

    for filename in os.listdir(image_dir):
        if filename.endswith(".png"):
            image_path = os.path.join(image_dir, filename)
            mask_path = None

            # Match the corresponding mask file
            base_name = filename.rsplit(".", 1)[0]
            for mask_file in os.listdir(mask_dir):
                tif_base_name = mask_file.replace("_root_mask", "").rsplit(".", 1)[0]
                if tif_base_name == base_name:
                    mask_path = os.path.join(mask_dir, mask_file)
                    break

            if not mask_path:
                print(f"Corresponding mask file not found for {filename}")
                skipped_files.append(filename)
                continue

            # Read images and masks
            image = cv2.imread(image_path)
            mask = cv2.imread(mask_path, cv2.IMREAD_UNCHANGED)

            if image is None or mask is None:
                print(f"Error reading image or mask for {filename}. Skipping.")
                skipped_files.append(filename)
                continue

            # Pad images and masks
            padded_image = padder(image, patch_size)
            padded_mask = padder(mask, patch_size)

            # Patchify images and masks
            image_patches = patchify(padded_image, (patch_size, patch_size, 3), step=patch_size)
            mask_patches = patchify(padded_mask, (patch_size, patch_size), step=patch_size)

            # Save patches
            for i in range(image_patches.shape[0]):
                for j in range(image_patches.shape[1]):
                    image_patch = image_patches[i, j, 0]
                    mask_patch = mask_patches[i, j]

                    # Create unique filenames for patches
                    image_patch_filename = f"{base_name}_patch_{i}_{j}.png"
                    mask_patch_filename = f"{base_name}_patch_{i}_{j}.tif"

                    # Save patches
                    cv2.imwrite(os.path.join(output_image_dir, image_patch_filename), image_patch)
                    cv2.imwrite(os.path.join(output_mask_dir, mask_patch_filename), mask_patch)

            print(f"Processed and patchified: {filename}")
            processed_files.append(filename)

    # Debugging information
    print("\nSummary of Patchifying:")
    print(f"Total files processed: {len(processed_files)}")
    print(f"Total files skipped: {len(skipped_files)}")
    if skipped_files:
        print("Skipped files:")
        for skipped in skipped_files:
            print(f"  - {skipped}")

# Define directories
image_dir = "D:/Holland_Year_2/Block_B/Task5_Patches/train_images_cropped_"
mask_dir = "D:/Holland_Year_2/Block_B/Task5_Patches/train_masks_cropped_"
output_image_dir = "D:/Holland_Year_2/Block_B/Task5_Patches/dataset_/train_images/train"
output_mask_dir = "D:/Holland_Year_2/Block_B/Task5_Patches/dataset_/train_masks/train"
# Ensure output directories exist
os.makedirs(output_image_dir, exist_ok=True)
os.makedirs(output_mask_dir, exist_ok=True)

# Patchify images and masks
patch_size = 256
patchify_images_and_masks(image_dir, mask_dir, output_image_dir, output_mask_dir, patch_size)


Processed and patchified: train_Alican_230858_im1.png
Processed and patchified: train_Alican_230858_im2.png
Processed and patchified: train_Alican_230858_im3.png
Processed and patchified: train_Alican_230858_im4.png
Processed and patchified: train_Alican_230858_im5.png
Processed and patchified: train_Alican_231131_im1.png
Processed and patchified: train_Alican_231131_im2.png
Processed and patchified: train_Alican_231131_im3.png
Processed and patchified: train_Alican_231131_im4.png
Processed and patchified: train_Alican_231131_im5.png
Processed and patchified: train_Alican_231648_im1.png
Processed and patchified: train_Alican_231648_im2.png
Processed and patchified: train_Alican_231648_im3.png
Processed and patchified: train_Alican_231648_im4.png
Processed and patchified: train_Alican_231648_im5.png
Processed and patchified: train_Alican_232922_im1.png
Processed and patchified: train_Alican_232922_im2.png
Processed and patchified: train_Alican_232922_im3.png
Processed and patchified: tr